In [1]:
from RSA.encryption import *
import collections

### Instantiate RSA class with primes p and q.  p has length 100 and q length 150. The exponent e used in encryption can have smaller length therefore we chose 10. 

In [2]:
rsa = RSA(100, 150, 10)

### Next we generate the encryption key and the decyption key. The public key N is shared with everyone and d is the private key we keep secret!

In [3]:
rsa.generate_key()

{'p': 5645476846671849093203667796127833182471626577225275193848121776547169693710793615307202016532524589,
 'q': 739956572690467304510922139235863336917648565835411337441598155420583639252069340303448206462508191565503130804906642298988860815399870180702758476747,
 'e': 5397499747,
 'N': 4177407698666688244908626643836844718925736775470715133570637235710058618937054056782828063614592149521311542277559019371138997549810824287557659209661833145239146879813337861773480513613504204451755263430026296592522753779086009854859710947962231983,
 'd': 2062965986399966437032456692149293565672500673322736994761964833054575301680023408681065280743816757925664084347630429600575241368793245984180625704831121407727038415682309191654926552713563452485641434030129387903753194052537159119187719290072963899}

In [4]:
messege = rsa.encrypt('This is a secret message.')

# The messege is now encrypted as a list of very large integers

In [5]:
messege

[821445262037521701187953745283591110499944797306202434249610897591901419520642642686407465747540693412448384615561958796771425577547961774696541118225906205514088492800727705656031581328918626592406598159788371820872244106530362977711779899818430613,
 3103195669394899990971227050832404532395011558146613265635197116631717101239579599028153256103322745174847962881030485779976286484316667669175523573511813618693655289156382593572234012347578736240841535593563051012582101767456305851492575726515493278,
 3121326881427529153522606901105809681074186275576740297443380138327407354987311931403493364131179073712407055588663726073839699332496280760775649556961396002853604126284530217350676790948531306597561534005273725522226827412004735126051305370876061241,
 9393263490939887748518577325727304606263121790809412306197716223671728583423718655094825831427429044731652726442703906802978592218734685266813998434240803900036654866373755536063811146408852533230890866298413867849759830967630080483217460411

### Below we look for duplicates. Since there are duplicates we should explore a more robust padding function. The goal with the padding is to avoid duplicate encoded numbers. There are 10 

In [6]:
dups = [item for item, count in collections.Counter(messege).items() if count > 1]
print(len(dups))
print(len(messege))

5
25


In [7]:
rsa.decrypt(messege)

'This is a secret message.'